In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Задача бинарной классификации. Метрика - accuracy

In [2]:
df = pd.read_csv("train.csv", index_col="PassengerId")
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Выделю target в отдельную переменную и удалю колонки, которые нам никак не помогут(имя пассажира, номер билета). Объединим количeство родственников в одну колонку

In [3]:
y = np.array(df["Survived"])

In [4]:
df['SibSp'] = df.loc[:,['SibSp','Parch']].sum(axis=1)
df = df.drop(["Name", "Ticket", "Parch"], axis=1)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Fare,Cabin,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,7.2500,NaN,S
2,1,1,female,38.0,1,71.2833,C85,C
3,1,3,female,26.0,0,7.9250,NaN,S
4,1,1,female,35.0,1,53.1000,C123,S
5,0,3,male,35.0,0,8.0500,NaN,S


### Смотрим сколько "Null" в данных

In [5]:
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

### Изменяю номер каюты на номер палубы (от 1 до 7, сверху вних)

In [6]:
arr_letter = ["A", "B", "C", "D", "E", "F", "G"]
for i in range(1, len(df) + 1):
    if not pd.isnull(df.loc[i]["Cabin"]) and df.loc[i]["Cabin"][0] in arr_letter:
        df.loc[i, "Cabin"] = float(arr_letter.index(df.loc[i, "Cabin"][0])) + 1.0
    else:
        df.loc[i, "Cabin"] = None

### Подсчитываю среднее для номера этажа, где расположена каюта и средний возраст, сортируя данные по классам билетов. Изменяю Null на соответствующие средние

In [7]:
cabin_class_dict = {}
age_class_dict = {}
for i in range(1, 4):
    cabin_class_dict[i] = round(df[(df["Pclass"] == i) & (df["Cabin"].notna())]["Cabin"].mean(), 2)
    age_class_dict[i] = round(df[(df["Pclass"] == i) & (df["Age"].notna())]["Age"].mean())
cabin_class_dict, age_class_dict

({1: 3.01, 2: 5.25, 3: 6.08}, {1: 38, 2: 30, 3: 25})

In [8]:
for i in range(1, 4):
    df.loc[(df["Pclass"] == i) & (df["Cabin"].isna()), "Cabin"] = cabin_class_dict[i]
    df.loc[(df["Pclass"] == i) & (df["Age"].isna()), "Age"] = age_class_dict[i]
df["Cabin"].isna().sum(), df["Age"].isna().sum()

(0, 0)

### Перевожу пол в числа, перевожу порт в числа, где 1 - это откуда корабль отправился в плавание, а 3 - последний порт, где он был

In [9]:
df.loc[df["Sex"] == "female", "Sex"] = 1
df.loc[df["Sex"] == "male", "Sex"] = 0

In [10]:
arr_port = ["S", "C", "Q"]
for i in range(3):
    df.loc[(df["Embarked"] == arr_port[i]) & (df["Embarked"].notna()), "Embarked"] = float(i + 1)

### В признаке "Embarked" остались Null. С помощью анализа выяслил, что пассажиры соответствующего класса с такой стоимостью билета вероятнее всего отправились из 1 порта

In [11]:
df[df["Embarked"].isna()]

,Survived,Pclass,Sex,Age,SibSp,Fare,Cabin,Embarked
PassengerId,,,,,,,,
62,1,1,1,38.0,0,80.0,2.0,NaN
830,1,1,1,62.0,0,80.0,2.0,NaN


In [12]:
for i in range(1, 4):
    print(i, "порт ~", df[(df["Pclass"] == 1) & (df["Cabin"] == 2) & (df["Embarked"] == i)]["Fare"].mean())

1 порт ~ 85.37228260869566
2 порт ~ 145.96401818181818
3 порт ~ nan


In [13]:
df.loc[df["Embarked"].isna(), "Embarked"] = 1.0
df.isna().sum().sum()

0

### От всех Null избавился, категориальные признаки закодировал. Осталось сделать нормализацию и подобрать модель

In [14]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Fare,Cabin,Embarked
PassengerId,,,,,,,,
1,0,3,0,22.0,1,7.2500,6.08,1.0
2,1,1,1,38.0,1,71.2833,3.0,2.0
3,1,3,1,26.0,0,7.9250,6.08,1.0
4,1,1,1,35.0,1,53.1000,3.0,1.0
5,0,3,0,35.0,0,8.0500,6.08,1.0


### Для удобства всю работу с данными положу в один класс

In [32]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import sklearn

class BaseDataPreprocessor(sklearn.base.TransformerMixin):
    def __init__(self):
        self.cabin_class_dict = {}
        self.age_class_dict = {}
        pass

    def fit(self, data, *args):
        for i in range(1, 4):
            self.cabin_class_dict[i] = round(df[(df["Pclass"] == i) & (df["Cabin"].notna())]["Cabin"].mean(), 2)
            self.age_class_dict[i] = round(df[(df["Pclass"] == i) & (df["Age"].notna())]["Age"].mean())
        return self

    def transform(self, df: pd.DataFrame) -> np.array:
        df['SibSp'] = df.loc[:,['SibSp','Parch']].sum(axis=1)
        df.loc[df["Sex"] == "female", "Sex"] = 1
        df.loc[df["Sex"] == "male", "Sex"] = 0
        df = df.drop(["Name", "Ticket"], axis=1)
        
        arr_letter = ["A", "B", "C", "D", "E", "F", "G"]
        for i in range(df.index[0], df.index[0] + len(df)):
            if not pd.isnull(df.loc[i]["Cabin"]) and df.loc[i]["Cabin"][0] in arr_letter:
                df.loc[i, "Cabin"] = float(arr_letter.index(df.loc[i, "Cabin"][0])) + 1.0
            else:
                df.loc[i, "Cabin"] = None

        for i in range(1, 4):
            df.loc[(df["Pclass"] == i) & (df["Cabin"].isna()), "Cabin"] = self.cabin_class_dict[i]
            df.loc[(df["Pclass"] == i) & (df["Age"].isna()), "Age"] = self.age_class_dict[i]

        arr_port = ["S", "C", "Q"]
        for i in range(3):
            df.loc[(df["Embarked"] == arr_port[i]) & (df["Embarked"].notna()), "Embarked"] = float(i + 1)
        df.loc[df["Embarked"].isna(), "Embarked"] = 1.0 # для train
        df_test.loc[df_test["Fare"].isna(), "Fare"] = df_test[(df_test["Pclass"] == 3) &
                                                              (df_test["Fare"].notna())]["Fare"].mean() # для test
        df = df.fillna(0)
        return np.array(df)

### Возьму несколько моделей для классификации, подберу для них параметры и оценю результат

In [33]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [34]:
df_train = pd.read_csv("train.csv", index_col="PassengerId")
df_test = pd.read_csv("test.csv", index_col="PassengerId")

In [35]:
preprocessor = BaseDataPreprocessor()
scaler = StandardScaler()
X_train, y_train = preprocessor.fit_transform(df_train.drop(["Survived"], axis=1)), np.array(df_train["Survived"])
X_train = scaler.fit_transform(X_train)

X_test = preprocessor.transform(df_test)
X_test = scaler.transform(X_test)

In [43]:
model = LogisticRegression(random_state=0).fit(X_train, y_train)
cross_val_score(model, X_train, y_train, cv=10).mean()

0.8047315855181024

In [42]:
parameters = {"criterion":["gini", "entropy", "log_loss"],
              "max_depth":[4, 5, 6],
              "min_samples_split":[2, 3, 4],
              "min_samples_leaf":[2, 3, 4]}
model = DecisionTreeClassifier()
grid = GridSearchCV(model, parameters, cv=10)
answer = grid.fit(X_train, y_train)
params = answer.best_params_
print(params)

model = DecisionTreeClassifier(**params)
cross_val_score(model, X_train, y_train, cv=10).mean()

{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 3, 'min_samples_split': 2}


0.813732833957553

In [41]:
parameters = {"n_estimators":[50, 100, 150],
              "criterion":["gini", "entropy"],
              "max_depth":[5, 6, 7],
              "min_samples_split":[2, 3, 4],
              "min_samples_leaf":[4, 5, 6]}

model = RandomForestClassifier()
grid = GridSearchCV(model, parameters, cv=10)
answer = grid.fit(X_train, y_train)
params = answer.best_params_
print(params)

model = RandomForestClassifier(**params)
cross_val_score(model, X_train, y_train, cv=10).mean()

{'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 50}


0.8305867665418226

In [40]:
parameters = {"loss_function":["Logloss"],
              "iterations":[300, 400, 500],
              "learning_rate":[0.05, 0.1],
              "depth":[3, 5, 7],
              "l2_leaf_reg":[1, 10]}

model = CatBoostClassifier(verbose=False)
grid = GridSearchCV(model, parameters, cv=10)
answer = grid.fit(X_train, y_train)
params = answer.best_params_
print(params)

model = CatBoostClassifier(**params, verbose=False)
cross_val_score(model, X_train, y_train, cv=10).mean()

{'depth': 5, 'iterations': 300, 'l2_leaf_reg': 1, 'learning_rate': 0.1, 'loss_function': 'Logloss'}


0.8440324594257177

### Catboost и случайный лес показали лучшие результаты. Создам 3 ответа с помощью: catboost, случайный лес, catboost+случайный лес

In [44]:
def save_pred(predict, name_file):
    answer = df_test.copy()
    answer["Survived"] = predict
    answer = answer["Survived"]
    answer.to_csv(name_file + ".csv")

In [45]:
params = {'depth': 5, 'iterations': 300, 'l2_leaf_reg': 1, 'learning_rate': 0.1, 'loss_function': 'Logloss'}
model = CatBoostClassifier(**params, verbose=False)
model.fit(X_train, y_train)
pred = model.predict(X_test)
save_pred(pred, "answer_catboost")

In [46]:
params = {'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 50}
model = RandomForestClassifier(**params)
model.fit(X_train, y_train)
pred = model.predict(X_test)
save_pred(pred, "answer_randomforest")

In [47]:
params = {'depth': 5, 'iterations': 300, 'l2_leaf_reg': 1, 'learning_rate': 0.1, 'loss_function': 'Logloss'}
model = CatBoostClassifier(**params, verbose=False)
model.fit(X_train, y_train)
pred_proba = model.predict_proba(X_test)

params = {'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 50}
model = RandomForestClassifier(**params)
model.fit(X_train, y_train)
pred_proba += model.predict_proba(X_test)

pred_proba /= 2
pred = []

for item in pred_proba:
    if item[1] > 0.5:
        pred.append(1)
    else:
        pred.append(0)
save_pred(pred, "answer_catboost_randomforest")

### После проверки лучшее качество получило предсказание, сделанное с помощью "RandomForestClassifier", 78%